In [10]:
import numpy as np
import pandas as pd
import math
from functools import reduce
import numpy as np
import pandas as pd
import os

In [11]:
def ent(x):
    '''
    :param x: 其中输入的x为一列数据
    :return: 该列数据的信息熵
    '''
    x=pd.Series(x)
    #定义初始信息熵ent1
    ent1=0
    p=x.value_counts()/len(x)
    for i in range(len(p)):
        e=-p.iloc[i]*math.log(p.iloc[i])
        ent1=ent1+e
    return ent1

In [12]:
def cutIndex(x,y):
    '''
    :param x: 其中输入的x为一列数据
    :param y: 标签数据
    :return: 该列数据的切点以及划分后的信息熵
    '''
    n=len(x)
    x=pd.Series(x)
    # 初始化一个熵的值
    entropy=9999
    cutD=None
    # 寻找最佳分裂点
    for i in range(n-1):
        if (x.iloc[i+1]!=x.iloc[i]):
            #cutX=(x.iloc[i+1]+x.iloc[i])/2
            wCutX=x[x<x.iloc[i+1]]
            wn=len(wCutX)/n
            # 左边权重wn
            e1=wn*ent(y[:len(wCutX)])
            # 右边权重
            e2=(1-wn)*ent(y[len(wCutX):])
            # 权重总和的信息熵
            val=e1+e2
            if val<entropy:
                entropy=val
                cutD=i
    if cutD==None:
        return(None)
    else:
        #返回切点,最小信息熵
        return(cutD,entropy)

In [13]:
#停止切分函数
def cutStop(cutD,y,entropy):
    '''
    :param cutD: 切点
    :param y: 标签列
    :param entropy:信息熵 
    :return: 信息熵的变化
    '''
    n=len(y)
    es=ent(y)#总信息熵
    gain=es-entropy#信息熵的变化
    left=len(set(y[0:cutD]))
    right=len(set(y[cutD:]))
    lengthY=len(set(y))
    if (cutD==None or lengthY==0):
        return (None)
    else:
        #math.log(3^2-2) 也就是math.log(7) 
        delta=math.log(3**lengthY-2)-(lengthY*ent(y)-left*ent(y[0:cutD])-right*ent(y[cutD:]))
        cond=math.log(n-1)/n+delta/n
    #大约选择变化小于总体信息熵的20%(例如:总体熵为1,当切点且分后变化小于0.2的时候不需要进行切分)
        if (gain < cond):
            return(None)
        else:
            return(gain)

In [14]:
#返回切点的位置
def cutPoints(x,y):
    '''
    :param x: 输入带切分的列数据
    :param y: 输入标签数据
    :return: 返回切点
    '''
    dx=x.sort_values()
    dy=pd.Series(y,index=dx.index)#按照X的排序更新y的值
    depth=0
    def gr(low,upp,depth=depth):
        x=dx[low:upp]
        y=dy[low:upp]
        n=len(y)
        k=cutIndex(x,y)
        if k==None:
            return(None)#判断是否存在切点 加权求和
        else:
            cutD=k[0]#切点位置
            entropy=k[1]#信息熵
            gain=cutStop(cutD,y,entropy)
            if gain==None:
                return(None)#判断是否应该切分(基于熵的变化)
            else:
                return[cutD,depth+1]
    #递归函数返
    def part(low=0,upp=len(dx),cutTd1=[],depth=depth):
        x1 = dx[low:upp]
        y1 = dy[low:upp]
        n = len(x1)
        # 返回的是切点 与depth+1
        k = gr(low,upp,depth=depth)
        if (n<2 or k==None):
            return cutTd1
        else:
            cutX=k[0]
            depth=depth+1
            cutTd1.append(low + cutX)
            #cutTd1.extend([low+cutX])
            # 从小到大
            cutTd1.sort()
            return part(low,low+cutX,cutTd1,depth)+part(cutX+low,upp,cutTd1,depth)
    res1=part(low=0,upp=len(dx),cutTd1=[],depth=depth)
    cutDx=[]
    if res1==[]:
        return(None)
    #去重
    func = lambda x, y: x if y in x else x + [y]
    res=reduce(func, [[], ] +res1)
    res=pd.Series(res)
    #返回切点的label 对应的实际值
    for i in res.values:
        k=round((x.sort_values().values[i]+x.sort_values().values[i+1])/2,6)
        cutDx.append(k)
    return(cutDx)

In [22]:
def Mdlp(data):
    '''
    :param data: 输入的是df 需要带有label字段
    :return: [切点] [min,切点,max]
    '''
    p=data.shape[1]-1
    y=data.iloc[:,p]
    xd=data
    cutP=[]
    cutPs=[]
    for i in range(0,p):
        x=data.iloc[:,i]
        cuts1=cutPoints(x,y)
        if cuts1==None:
            cuts1="ALL"
        cuts=[[min(x)],cuts1,[max(x)]]
        cutPs.append(cuts)
        cutP.append(cuts1)
        #label=range(1,len(cuts))
        #xd.ix[:,i]=pd.cut(x,bins=cutpc,labels=label,include_lowest=True)
    return cutP,cutPs

In [27]:
def Mdlpx(data):
    '''
    :param data: 输入的是df 需要带有label字段
    :return:  {column_name:切点} {column_name:min,切点,max}
    '''
    cut,cutAll=Mdlp(data)
    ziDian={}
    ziDian_all={}
    for i in range(data.shape[1]-1):
        ziDian[data.columns[i]]=cut[i]
        ziDian_all[data.columns[i]]=cutAll[i]
    return ziDian,ziDian_all

In [28]:
df = pd.read_csv('original_data.csv')
df

,pos x,pos y,vel x,vel y,angle,angleVel,contact_1,contact_2,action
0,-0.007577,1.408559,-7.674955e-01,-1.049699e-01,0.008787,1.738492e-01,0,0,3
1,-0.015080,1.405617,-7.570502e-01,-1.308139e-01,0.015491,1.341034e-01,0,0,3
2,-0.022517,1.402089,-7.488564e-01,-1.568784e-01,0.020541,1.010064e-01,0,0,3
3,-0.029875,1.397969,-7.388699e-01,-1.831327e-01,0.023582,6.082692e-02,0,0,3
4,-0.037145,1.393242,-7.278200e-01,-2.101228e-01,0.024409,1.654167e-02,0,0,3
...,...,...,...,...,...,...,...,...,...
4460,0.033173,-0.001067,5.220000e-08,2.800000e-10,0.001681,-8.620000e-08,1,1,0
4461,0.033173,-0.001067,-1.430000e-08,-5.730000e-11,0.001681,2.530000e-08,1,1,0
4462,0.033173,-0.001067,-4.650000e-08,-5.460000e-11,0.001681,9.920000e-08,1,1,0
4463,0.033173,-0.001067,-4.530000e-08,-5.450000e-11,0.001681,8.420000e-08,1,1,0


In [29]:
df_1 = Mdlpx(df)

In [30]:
df_1

({'pos x': [-0.379371,
   -0.144778,
   -0.137077,
   -0.136728,
   -0.088445,
   -0.049557,
   -0.04953,
   -0.03123,
   -0.030964,
   -0.006861,
   0.009555,
   0.009564,
   0.01084,
   0.010944,
   0.032404,
   0.033163,
   0.03332],
  'pos y': [-0.001068,
   -0.000506,
   -0.000504,
   -7.9e-05,
   -2.5e-05,
   0.064895,
   0.172832,
   0.46599,
   1.378156,
   1.098675,
   1.338458,
   1.43267],
  'vel x': [-0.720271,
   -0.652289,
   -0.625996,
   -0.481929,
   -0.168358,
   -0.004367,
   0.17765,
   -0.000122,
   9.7e-05,
   0.005651,
   0.340448],
  'vel y': [-0.367982,
   -0.183108,
   -0.005961,
   -0.000488,
   -1e-06,
   0.000396,
   -0.0,
   0.0,
   0.077104,
   0.210959],
  'angle': [-0.416013,
   -0.113565,
   -0.005627,
   0.000314,
   0.003823,
   0.007769,
   0.027345],
  'angleVel': [-0.16712,
   -0.07963,
   0.044889,
   -0.000323,
   -9.2e-05,
   4.5e-05,
   0.000992,
   0.148894],
  'contact_1': [0.5],
  'contact_2': [0.5]},
 {'pos x': [[-0.525974751],
   [-0.3793